### Notebook to calculate quantities for wind ensemble plot.
Specifically, horizontal salinity gradient magnitude, vertical salinity gradient magnitude, and vertical salinity diffusivity

In [6]:
#Packages 
import numpy as np
import xgcm
from xgcm import Grid
import xarray as xr
import xroms
from datetime import datetime
import glob
from datetime import timedelta
 #The chaotic option, used to suppress issues with cf_time with xroms 
import warnings
warnings.filterwarnings("ignore")

In [8]:
paths = ['/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_001/shelf_dx_500_uwind_osc_001_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_002/shelf_dx_500_uwind_osc_002_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_004/shelf_dx_500_uwind_osc_004_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_008/shelf_dx_500_uwind_osc_008_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_01/shelf_dx_500_uwind_osc_01_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_02/shelf_dx_500_uwind_osc_02_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_04/shelf_dx_500_uwind_osc_04_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_08/shelf_dx_500_uwind_osc_08_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_09/shelf_dx_500_uwind_osc_09_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_1/shelf_dx_500_uwind_osc_1_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_2/shelf_dx_500_uwind_osc_2_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_4/shelf_dx_500_uwind_osc_4_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_8/shelf_dx_500_uwind_osc_8_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_10/shelf_dx_500_uwind_osc_10_avg.nc',
         '/d1/shared/shelfstrat_wind/43N_open/shelf_dx_500_uwind_osc_10p5/shelf_dx_500_uwind_osc_10p5_avg.nc']

#Open model output with xroms
def open_roms(path):
    ds = xroms.open_netcdf(path)
    ds, grid = xroms.roms_dataset(ds)
    # Add 4 days to account for the change in initial condition b.c. ROMS counts at 1, NOT ZERO
    ds['ocean_time'] = ds.ocean_time[:]+timedelta(days=4)
    return ds, grid

ds = []
grid = []
for i in range(len(paths)):
    dsa, grida = open_roms(paths[i])
    ds.append(dsa)
    grid.append(grida)

In [16]:
etaslice = slice(1,100)
xislice = slice(1,-1)
tslice = slice(60+24,241+24) #Days 7.5-15
tslice1 = slice(0+24,360+24) #Days 5-20

time = np.arange(0,len(ds[0].ocean_time))/24 #Crude time vector, could do this a fancier way but meh. 
time+=5
print(time[0])
print(time[-1])
print(ds[0].ocean_time[0].values)
print(ds[0].ocean_time[-1].values)

5.0
21.958333333333332
0001-01-05 00:30:00
0001-01-21 23:30:00


In [17]:
def sgradmag_mean(ds, grid, tslice, etaslice, xislice):
    '''
Computes the horz. salinity gradient magnitude for the entire
water column on the psi points.
    ''' 
    dsaltdxi, dsaltdeta = xroms.hgrad(ds.salt, grid)
    dsaltdxi_rho = xroms.to_rho(dsaltdxi, grid)
    dsaltdeta_rho = xroms.to_rho(dsaltdeta, grid)
    sgradmag = (dsaltdxi_rho**2+dsaltdeta_rho**2)**(1/2)
    sgradmag_srho = grid.interp(sgradmag, 'Z', boundary = 'extend')
    smean = sgradmag_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).mean(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
    # smedian = sgradmag_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).median(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
    sstd = sgradmag_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).std(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
    return smean, sstd

def dsdz(ds, grid, tslice, etaslice, xislice):
    '''
Computes the vertical salinity gradient magnitude for the entire
water column on the psi points.
    ''' 
    dsdz = xroms.ddz(ds.salt, grid)
    dsdz_srho = grid.interp(dsdz, 'Z', boundary = 'extend')
    dsdz_srho.attrs = ''
    dsdz_srho.name = 'dsdz'
    svmean = dsdz_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).mean(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
    svstd = dsdz_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).std(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
   
    return svmean, svstd

def vert_diffusivity(ds, grid, tslice, etaslice, xislice):
    '''
Computes the vertical salinity gradient magnitude for the entire
water column on the psi points.
    ''' 
    aks = ds.AKs
    aks_srho = grid.interp(aks, 'Z', boundary = 'extend')
    aks_srho.attrs = ''
    aks_srho.name = 'aks'
    aksmean = aks_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).mean(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
    aksstd = aks_srho.isel(ocean_time = tslice, eta_rho = etaslice, xi_rho = xislice).std(['ocean_time', 'xi_rho', 's_rho', 'eta_rho']).values
   
    return aksmean, aksstd

### Run functions and save to a netcdf

In [19]:
s = []
s_std = []
for i in range(len(paths)):
    print(i)
    # smean, sstd = sgradmag_mean(ds[i], grid[i], tslice, etaslice, xislice) #Day 7.5-15
    smean, sstd = sgradmag_mean(ds[i], grid[i], tslice1, etaslice, xislice) #Day 5-20
    s.append(smean)
    s_std.append(sstd)
    
# np.save('outputs/wind_ens/mean_shgrad_allsims_day7p5_15', s)
# np.save('outputs/wind_ens/std_shgrad_allsims_day7p5_15', s_std)
np.save('outputs/wind_ens/mean_shgrad_allsims_day5_20', s)
np.save('outputs/wind_ens/std_shgrad_allsims_day5_20', s_std)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [20]:
sv = []
svd = []
for i in range(len(paths)):
    # svmean, svstd = dsdz(ds[i], grid[i], tslice, etaslice, xislice)
    svmean, svstd = dsdz(ds[i], grid[i], tslice1, etaslice, xislice)
    sv.append(svmean)
    svd.append(svstd)
    
# np.save('outputs/wind_ens/mean_svgrad_allsims_day7p5_15', sv)
# np.save('outputs/wind_ens/std_svgrad_allsims_day7p5_15', svd)
np.save('outputs/wind_ens/mean_svgrad_allsims_day5_20', sv)
np.save('outputs/wind_ens/std_svgrad_allsims_day5_20', svd)

In [23]:
diffu = []
diffustd = []
for i in range(len(paths)):
    # aksmean, aksstd = vert_diffusivity(ds[i], grid[i], tslice, etaslice, xislice)
    aksmean, aksstd = vert_diffusivity(ds[i], grid[i], tslice1, etaslice, xislice)
    diffu.append(aksmean)
    diffustd.append(aksstd)
    
# np.save('outputs/wind_ens/mean_sdfiffusivity_allsims_day7pt_15', diffu)
# np.save('outputs/wind_ens/std_sdfiffusivity_allsims_day7pt_15', diffustd)
np.save('outputs/wind_ens/mean_sdfiffsuvity_allsims_day5_20', diffu)
np.save('outputs/wind_ens/std_sdfiffsuvity_allsims_day5_20', diffustd)